In [1]:
import numpy as np

# This is the Q Matrix from the Yang book
## The reason for using the Q matrix from the Yang book is because we know that the manipulations done to the Q Matrix in the Yang book should perform as expected and if it can function properly it will show that the issue is not within the code but out Q Matrix its self. 

In [5]:
Q = np.mat("-1.4651 1.3374 0.1277 0; 1.2154 -1.2220 0.0066 0; 0.0099 0.0808 -0.5993 0.5086; 0 0 0.8530 -0.8530")
Q

matrix([[-1.4651,  1.3374,  0.1277,  0.    ],
        [ 1.2154, -1.222 ,  0.0066,  0.    ],
        [ 0.0099,  0.0808, -0.5993,  0.5086],
        [ 0.    ,  0.    ,  0.853 , -0.853 ]])

In [3]:
def expected_number_of_transitions_in_time_t1(t, Q):  
    m= Q.shape[0]
    eigenvalue,eigenvector = np.linalg.eig(Q)
    eigenvector_inv = np.linalg.inv(eigenvector)
    E_n = np.zeros((m,m))
    for i in range(m):      
        d_i=eigenvalue[i]       
        E_i = np.zeros((m,m))
        E_i[i,i]=1
        S_i = np.matmul(eigenvector,(np.matmul(E_i,eigenvector_inv)))
        for j in range(m):
            d_j=eigenvalue[j]
            E_j = np.zeros((m,m))
            E_j[j,j]=1
            S_j= np.matmul(eigenvector,(np.matmul(E_j,eigenvector_inv)))
            Q_l = np.zeros((m,m))
            Q_l[i,j] = Q[i,j]
            if d_i == d_j:
                I_ij = t*np.exp(d_i*t)
            else: 
                I_ij = (np.exp(d_i*t)-np.exp(d_j*t))/(d_i - d_j)
            N_greek= S_i@Q_l@S_j*I_ij
            E_n += N_greek
    return E_n

In [6]:
expected_number_of_transitions_in_time_t(1, Q)

array([[-0.0297251 , -0.09184213,  0.06620335, -0.02594866],
       [-0.07446889, -0.27301061, -0.08997005, -0.13062929],
       [ 0.06747336, -0.06023331, -0.27318334, -0.02335298],
       [-0.02353135, -0.15899654, -0.0490429 , -0.06751508]])

# ^ As you can see, the matrix spits out a bunch of negative values which should be representing the expected number of transitions. ^ 

# Below are the other strategies I used to try to produce the Q Matrix with the expected number of transitions

# ----------------------------------------------------------------------------------------------------------

# Below is the original Set-up of the computations to calculate the number of transitions

In [7]:
def expected_number_of_transitions_in_time_t2(t, Q):  
    m= Q.shape[0]
    eigenvalue,eigenvector = np.linalg.eig(Q)
    eigenvector_inv = np.linalg.inv(eigenvector)
    E_n = np.zeros((m,m))
    for i in range(m):
        d_i=eigenvalue[i]
        E_i = np.zeros((m,m))
        E_i[i,i]=1
        S_i = eigenvector*E_i*eigenvector_inv
        for j in range(m):
            d_j=eigenvalue[j]
            E_j = np.zeros((m,m))
            E_j[j,j]=1
            S_j= eigenvector*E_j*eigenvector_inv
            if d_i == d_j:
                I_ij = t*np.exp(d_i*t)
            else: 
                I_ij = (np.exp(d_i*t)-np.exp(d_j*t))/(d_i - d_j)
            N_greek=S_i*Q*S_j*I_ij
            E_n += N_greek
    return E_n

In [10]:
expected_number_of_transitions_in_time_t2(3, Q)

array([[-0.05943374, -0.06309743,  0.07230058,  0.05023059],
       [-0.06197969, -0.0722013 ,  0.08161735,  0.05256364],
       [ 0.05418001,  0.05934399, -0.09175121, -0.02177279],
       [ 0.05879995,  0.06793525, -0.0365163 , -0.09021891]])

# In the example above, 2 things are wrong. 
## Using an asterick for matrix multiplication is wrong, there is a better way to multiply matrices. 
## The Expected Number of Transitions Matrix in Time t, has negatives in the diagonal and has negative in various ij states when evaluated for 3< years. 

In [13]:
def expected_number_of_transitions_in_time_t3(t, Q):  
    m= Q.shape[0]
    eigenvalue,eigenvector = np.linalg.eig(Q)
    eigenvector_inv = np.linalg.inv(eigenvector)
    E_n = np.zeros((m,m))
    for i in range(m):
        d_i=eigenvalue[i]
        E_i = np.zeros((m,m))
        E_i[i,i]=1
        S_i = eigenvector@E_i@eigenvector_inv
        for j in range(m):
            d_j=eigenvalue[j]
            E_j = np.zeros((m,m))
            E_j[j,j]=1
            S_j= eigenvector@E_j@eigenvector_inv           
            if d_i == d_j:
                I_ij = t*np.exp(d_i*t)
            else: 
                I_ij = (np.exp(d_i*t)-np.exp(d_j*t))/(d_i - d_j)
            N_greek=S_i@Q@S_j*I_ij
            E_n += N_greek         
    return E_n

In [32]:
expected_number_of_transitions_in_time_t3(3, Q)

array([[-0.05943374, -0.06309743,  0.07230058,  0.05023059],
       [-0.06197969, -0.0722013 ,  0.08161735,  0.05256364],
       [ 0.05418001,  0.05934399, -0.09175121, -0.02177279],
       [ 0.05879995,  0.06793525, -0.0365163 , -0.09021891]])

# ^ Even when implementing the correct, matrix multipilcation function, the^ same Expected Number of Transitions in Time t Matrix is produced. Odd  

In [34]:
def expected_number_of_transitions_in_time_t4(t, Q):  
    m= Q.shape[0]
    eigenvalue,eigenvector = np.linalg.eig(Q)
    eigenvector_inv = np.linalg.inv(eigenvector)
    E_n = np.zeros((m,m))
    for i in range(m):
        d_i=eigenvalue[i]
        E_i = np.zeros((m,m))
        E_i[i,i]=1
        S_i = np.matmul(eigenvector,(np.matmul(E_i,eigenvector_inv)))        
        for j in range(m):
            d_j=eigenvalue[j]
            E_j = np.zeros((m,m))
            E_j[j,j]=1
            S_j= np.matmul(eigenvector,(np.matmul(E_j,eigenvector_inv)))          
            if d_i == d_j:
                I_ij = t*np.exp(d_i*t)
            else: 
                I_ij = (np.exp(d_i*t)-np.exp(d_j*t))/(d_i - d_j)
            N_greek=np.matmul((np.matmul(S_i,Q)),S_j)*I_ij
            E_n += N_greek        
    return E_n

In [35]:
expected_number_of_transitions_in_time_t4(1, Q)

array([[-0.12876896,  0.07214498,  0.03646152,  0.02016245],
       [ 0.06325738, -0.11630073,  0.04061239,  0.01243096],
       [ 0.02698167,  0.02983185, -0.16848619,  0.11167267],
       [ 0.01453499,  0.02473099,  0.18729216, -0.22655813]])

# This last example is to prove that the @ symbol and the np.matmul function operate the same and it doesn't matter which one is used. As displayed above the same Expected Number of Transitions in Time t Matrix is produced.  

# My next step is to try everything over again, but with Scipy. It seems that some things on Scipy work a bit better than some things on numpy. 

In [22]:
import scipy
from scipy import linalg

In [36]:
def expected_number_of_transitions_in_time_t5(t, Q):  
    m= Q.shape[0]
    eigenvalue,eigenvector = linalg.eig(Q)
    eigenvector_inv = linalg.inv(eigenvector)
    E_n = np.zeros((m,m))
    for i in range(m):
        d_i=eigenvalue[i]
        E_i = np.zeros((m,m))
        E_i[i,i]=1
        S_i = np.matmul(eigenvector,(np.matmul(E_i,eigenvector_inv)))        
        for j in range(m):
            d_j=eigenvalue[j]
            E_j = np.zeros((m,m))
            E_j[j,j]=1
            S_j= np.matmul(eigenvector,(np.matmul(E_j,eigenvector_inv)))          
            if d_i == d_j:
                I_ij = t*np.exp(d_i*t)
            else: 
                I_ij = (np.exp(d_i*t)-np.exp(d_j*t))/(d_i - d_j)
            N_greek=np.matmul((np.matmul(S_i,Q)),S_j)*I_ij
            np.add(E_n, N_greek, out=E_n, casting="unsafe")
    return E_n

In [37]:
expected_number_of_transitions_in_time_t5(3, Q)

C:\Users\simon\Anaconda3\Lib\site-packages\ipykernel_launcher.py:21: ComplexWarning: Casting complex values to real discards the imaginary part


array([[-0.05943374, -0.06309743,  0.07230058,  0.05023059],
       [-0.06197969, -0.0722013 ,  0.08161735,  0.05256364],
       [ 0.05418001,  0.05934399, -0.09175121, -0.02177279],
       [ 0.05879995,  0.06793525, -0.0365163 , -0.09021891]])

# Like all the others, it still produces the same Expected Number of Transitions Q Matrix and it also has this weird error thing.  

### P.S. I could not find a matrix multiplication function in Scipy. If you know of any, please let me know.